# Task 1.6D: Data Mining Challenge

* **Name**: Tri Khuong Nguyen
* **Student number**: 224605002
* **Email address**: s224605002@deakin.edu.au
* I am an *undergraduate (SIT220)* student

## Abstract

This report includes importing and merging the datasets downloaded from **NHANES**, handling missing values from the merged data frame, creating 5 interactive charts visualising the executed data, and covering data privacy issues that arise from these activities.

## Importing packages

* First, we must import the packages we will use throughout this task.

* `numpy` and `pandas` are used for computing purposes and executing data frames (`pandas` exclusively).

* For this task, we need to use the `bokeh` package to plot interactive charts - particularly 4 modules `bokeh.plotting` with methods `figure`, `show`, `output_notebook`; `bokeh.layouts` with methods `row`, `column`; `bokeh.transform` with method `transform`; `bokeh.models` with methods `ColumnDataSource`, `HoverTool`, `CustomJS`, `Select`, `Dropdown`, `Slider`, `ColorBar`, `LinearColorMapper` (The details of these methods will be covered when we apply them). An extra module `bokeh.palettes` is used for generating the color palettes for the last visualisation.

In [4]:
import numpy as np
import pandas as pd

from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import row, column
from bokeh.transform import transform
from bokeh.models import (ColumnDataSource, HoverTool, CustomJS, Select,
                          Dropdown, Slider, ColorBar, LinearColorMapper)
import bokeh.palettes


$\rightarrow$ Next, we will load our data.


## Loading datasets

* We download the 5 datasets from the [**`NHANES` August 2021-August 2023**](https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?Cycle=2021-2023) - [*Body Measures (as `body_measures.xpt`)*](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/BMX_L.xpt), [*Current Health Status (as `current_health_status.xpt`)*](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/HSQ_L.xpt), [*Dietary Interview - Total Nutrient Intakes, First Day (as `dietary_totalnutrient_firstday.xpt`)*](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/DR1TOT_L.xpt), [*Cholesterol - Total (as `total_cholesterol.xpt`)*](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/TCHOL_L.xpt), [*Blood Pressure - Oscillometric Measurements (as `blood_pressure.xpt`)*](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/BPXO_L.xpt), then respectively load them into data frames `body_measures`, `health_status`, `dietary_nutrient`, `total_cholesterol`, `blood_pressure` using `pd.read_sas()`.

In [7]:
# Load datasets
body_measures = pd.read_sas('body_measures.xpt')
health_status = pd.read_sas('current_health_status.xpt')
dietary_nutrient = pd.read_sas('dietary_totalnutrient_firstday.xpt')
total_cholesterol = pd.read_sas('total_cholesterol.xpt')
blood_pressure = pd.read_sas('blood_pressure.xpt')


$\rightarrow$ The data are loaded into our notebook.


## Merging data frames

* The imported data frames can be cumbersome, and conventionally, we want to merge them into a single data frame, named `main_df`.

* To merge the data frames, we use outer join on their shared `SEQN` column (contains respondent sequence numbers that uniquely identify each survey participant). An outer join of data frame `a` and data frame `b` on column `SEQN` means each row in `a` is merged with another row in `b` (concatenating columns from both data frames) if they share the same `SEQN`. If a `SEQN` value exists at `n` rows in `a` and `m` rows in `b`, we will have `n x m` rows in the result which are combinations from `n` and `m`, and if a `SEQN` value found in `a` does not match any row(s) in `b` or vice versa then the concatenation will still happen but the values from the columns of the data frame that has missing compared `SEQN` are set to `NaN`.

* For the first outer join, we merge `body_measures` with `health_status` (`body_measures.merge( health_status)`) on matching `SEQN` (`on="SEQN"`) and by outer join (`how="outer"`). Whatever the current data frame is, we implement the second outer join, merging it with `dietary_nutrient` (`.merge( dietary_nutrient)`) on matching `SEQN` (`on="SEQN"`) and by outer join (`how="outer"`). The logic applies for the remaining data frames (`total_cholesterol` and `blood_pressure`).

In [10]:
# Merge data frames on "SEQN" column using outer join
main_df = (body_measures
           .merge(health_status, on="SEQN", how="outer")
           .merge(dietary_nutrient, on="SEQN", how="outer")
           .merge(total_cholesterol, on="SEQN", how="outer")
           .merge(blood_pressure, on="SEQN", how="outer"))


$\rightarrow$ With outer join, having missing values (`NaN`) in `main_df` is inevitable, and we obviously want to eliminate these.


## Missing value handling

* There are many methods for handling missing values, but in our case, for every column (`for col in main_df.columns`), we have 3 principles: if all values in a column are completely missing (`if main_df[col].isna().all()`), we fill them with the string `"No data"` (assign that to `fill_value`) because there are no existing values for us to impute the missing ones; else if that column has number as the data type (`elif main_df[col].dtype == 'float64'`), we fill the missing values with the median of existing values in that column (assigning `main_df[col].median()` to `fill_value`) since it is a neutral value that has a low chance of distorting our dataset; else for a non-numeric or string-type column, its missing values should be filled with its mode (assigning `main_df[col].mode()[0]` to `fill_value`, with `.mode()[0]` returns the first mode that we pay attention to) as that is the most common value and the safest to fill.

* After identifying the common fill value (`fill_value`) for the column, we begin filling the missing values with `fill_value` using the `fillna()` method and overwriting the old column (`main_df[col] = main_df[col].fillna(fill_value)`).

In [13]:
# Handle missing values (missing all, num-type, string-type)
for col in main_df.columns:
    if main_df[col].isna().all():
        fill_value = "No data"
    elif main_df[col].dtype == 'float64':
        fill_value = main_df[col].median()
    else:
        fill_value = main_df[col].mode()[0]
    
    main_df[col] = main_df[col].fillna(fill_value)


$\rightarrow$ Our main data frame (`main_df`) is reformatted and ready for graphing visualisations.


## Declaring the output of `bokeh` visualisations

* As we want to output the `bokeh` visualisations directly into our notebook (as cells), we can simply declare `output_notebook()`.

In [16]:
output_notebook()

Loading BokehJS ...


$\rightarrow$ Now we are ready to create our visualisations.


## Scatter chart visualising pairs of body measures

* **Idea:** We want to create a scatter chart that illustrates the relationship between all pairs of numerical variables of the [*Body Measures*](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/BMX_L.htm) dataset. Only one pair is shown at a time (on the horizontal and vertical axes) but we can change the variables on the axes using dropdown menus.

* The numerical variables are originally in code form (`BMXWT`, `BMXRECUM`, `BMXHEAD`, `BMXHT`, `BMXBMI`, `BMXLEG`, `BMXARML`, `BMXARMC`, `BMXWAIST`, `BMXHIP`), and for a more meaningful presentation, we change these to their associated names (as per this [Codebook](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/BMX_L.htm#Codebook)) and store in a dictionary `mapping`.

* Next, we extract the columns of numerical variables from `main_df` (`main_df[["BMXWT", "BMXRECUM", "BMXHEAD", "BMXHT", "BMXBMI", "BMXLEG", "BMXARML", "BMXARMC", "BMXWAIST", "BMXHIP"]]`), rename those columns via the created `mapping` (`.rename(columns=mapping)`), and store them in a sub data frame `bodymeasures_df`. This will be used when we create the chart.

* For the creation of the chart, we first create a figure `scatter_chart` with the `figure()` function (imported from `bokeh.plotting`) with the title of two default variables `Weight (kg)` and `BMI (kg/m^2)` (`title="Relationship between Weight (kg) and BMI (kg/m^2)"`), and the dimension *600px x 400px* (`width=600, height=400`). Then, we use `scatter_chart.scatter()` to add the scatter points - with `source=ColumnDataSource(bodymeasures_df)` converts the sub data frame into a format that fits `bokeh` (each column in the data frame is a key-value pair where key is represented by column name and value is an array of records stored in that column); from `source`, the data of default variables `Weight (kg)` and `BMI (kg/m^2)` on the x-axis and y-axis respectively (`x="Weight (kg)", y="BMI (kg/m^2)"`); each point has the circle shape (`marker="circle"`), the size *5* (`size=5`), the *navy* color (`color="navy"`), and *40%* transparency (`alpha=0.4`) for clearer viewing of the overlapping points.

* The next step is creating the dropdown menus for changing variables of two axes. To do this, we first get the list of variables `labels` (column labels) by extracting the values from the `mapping` dictionary and converting them to a list (`list(mapping.values())`). Now we are ready to create the dropdown menus - `select_x` as the one for the x-axis (using imported `Select` from `bokeh.models` with the shown name `title="Horizontal axis"`, options `options=labels` available for selecting are variables stored in `labels` list, and the default variable `value="Weight (kg)"`); `select_y` uses the same model `Select` and the same variables (`options=labels`), with differences only in the name (`title="Vertical axis"`) and default variable (`value="BMI (kg/m^2)"`).

* We arrange the created components with the imported `row` and `column` from `bokeh.layouts` - `row(select_x, select_y)` generates a row with `select_x` on the left and `select_y` on the right, this row is placed on the top of `scatter_chart`, so they are put in a column (`column()`). The whole group is assigned to `layout`.

* Our chart already has some default interactive tools from `bokeh` by default, such as *Pan (move the chart around)*, *Box Zoom (draw a box to zoom into the drawn section)*, *Wheel Zoom (zoom in/out by scrolling the mouse wheel up/down)*, *Save (save the graph as image)*, *Reset (return to the original view after moving or zooming)*. However, we also want to add a hover tool that displays the information of a point (in a placeholder) when we hover on it. Our object is `hover_tool`, created using `HoverTool` model from `bokeh.models` with `tooltips` parameter receives a list of information to be displayed - the x-axis label followed by the x-value of the point (`("Weight (kg)", "@{Weight (kg)}")` as our default x-variable), and the same thing goes for y-label and y-value (`("BMI (kg/m^2)", "@{BMI (kg/m^2)}")` as our default y-variable). We then add our `hover_tool` to the chart (`scatter_chart.add_tools(hover_tool)`).

* The second interactive function that we are going to implement is changing data (using the dropdown menus). We have created the 2 dropdown menus, but have yet to link them to our chart, and to do that, we must first create a connection named `update_scatter_chart`, which is the [`CustomJS`](https://docs.bokeh.org/en/latest/docs/user_guide/interaction/js_callbacks.html#customjs-callbacks) method from `bokeh.models`. In `CustomJS()`, we must write a *JavaScript* code that performs our desired logic, consisting of 2 parts. For the first part, we map (pass) the variables created in *Python* (`select_x`, `select_y`, `scatter_chart`, `hover_tool`) to *JavaScript* through the dictionary of `args=dict()` (we keep the same name for every variable between 2 languages to avoid confusion). For the second part, we write *JavaScript* code inside `code=""" """`. To change the x-variable, we access `scatter_chart.renderers[0]` (the first and only renderer, which are the scatter points), then `.glyph` (refers to the circle markers of the points), and until `.x` (the x-coordinates of the points) that we set it to the new x-coordinates (the values stored in the column specified by `select_x.value`, which is the choice that we selected for the *Horizontal axis* dropdown menu); similarly, `scatter_chart.renderers[0].glyph.y = { field: select_y.value };` sets the values (`field`) of the selected variable (`select_y.value`) as the new y-coordinates. Additionally, we access the title `scatter_chart.title`, and `.text` to edit the text to a new title involving new x-variable name (`${select_x.value}`) and new y-variable name (`${select_y.value}`) as chosen on `select_x` and `select_y` dropdown menus. We also want to update `hover_tool` by accessing `.tooltips` and editing the elements - new x-axis label followed by the new x-value ``[`${select_x.value}`, `@{${select_x.value}}`]``, and new y-axis label followed by the new y-value ``[`${select_y.value}`, `@{${select_y.value}}`]``; `scatter_chart.add_tools(hover_tool)` applies the changes to the chart.

* Now, we can declare the connection from `select_x` and `select_y` to `scatter_chart` through `js_on_change()` method with `"value"` representing the changes in the value(s) of `select_x` and/or `select_y` respectively, and both use the same logic `update_scatter_chart`. This means every time a dropdown menu (`select_x` or `select_y`) receives a new value, the code in `update_scatter_chart` is executed.

* Lastly, we use `show()` (from imported `bokeh.plotting`) to display our arranged `layout` (`show(layout)`).

In [19]:
# Mapping from column codes to names
mapping = {"BMXWT": "Weight (kg)", "BMXRECUM": "Recumbent Length (cm)",
           "BMXHEAD": "Head Circumference (cm)", "BMXHT": "Height (cm)",
           "BMXBMI": "BMI (kg/m^2)", "BMXLEG": "Upper Leg Length (cm)",
           "BMXARML": "Upper Arm Length (cm)", "BMXARMC": "Arm Circumference (cm)",
           "BMXWAIST": "Waist Circumference (cm)", "BMXHIP": "Hip Circumference (cm)"}

# Data frame for the chart
bodymeasures_df = (main_df
                   [["BMXWT", "BMXRECUM", "BMXHEAD", "BMXHT", "BMXBMI",
                     "BMXLEG", "BMXARML", "BMXARMC", "BMXWAIST", "BMXHIP"]]
                   .rename(columns=mapping)
                  )

# Create scatter chart
scatter_chart = figure(title="Relationship between Weight (kg) and BMI (kg/m^2)",
                       width=600, height=400
                      )
scatter_chart.scatter(source=ColumnDataSource(bodymeasures_df),
                      x="Weight (kg)", y="BMI (kg/m^2)",
                      marker="circle", size=5, color="navy", alpha=0.4,
                     )

# Get the list of labels
labels = list(mapping.values())

# Dropdown menus
select_x = Select(title="Horizontal axis", options=labels, value="Weight (kg)")
select_y = Select(title="Vertical axis", options=labels, value="BMI (kg/m^2)")

# Arrange the components using a layout
layout = column(
                row(select_x, select_y),
                scatter_chart
               )

# Add hover tool
hover_tool = HoverTool(tooltips=[
    ("Weight (kg)", "@{Weight (kg)}"),
    ("BMI (kg/m^2)", "@{BMI (kg/m^2)}")
])
scatter_chart.add_tools(hover_tool)

# Logic to update scatter chart
update_scatter_chart = CustomJS(
    args=dict(select_x=select_x, select_y=select_y,
              scatter_chart=scatter_chart, hover_tool=hover_tool),
    code="""
    scatter_chart.renderers[0].glyph.x = { field: select_x.value };
    scatter_chart.renderers[0].glyph.y = { field: select_y.value };
    
    scatter_chart.title.text = `Relationship between ${select_x.value} ` +
                               `and ${select_y.value}`;
    
    hover_tool.tooltips = [
        [`${select_x.value}`, `@{${select_x.value}}`],
        [`${select_y.value}`, `@{${select_y.value}}`]
    ];
    scatter_chart.add_tools(hover_tool);
    """
)

# Call the implemented update function whenever the dropdown menu(s) is/are changed
select_x.js_on_change("value", update_scatter_chart)
select_y.js_on_change("value", update_scatter_chart)

# Display using show(layout)

In [20]:
show(layout)


$\rightarrow$ Out of all pairs of variables, we are most interested in the "Weight-BMI" pair as these variables are correlated, specifically directly proportional to each other via the *BMI formula* ($weight / height^2$ with weight in kilograms and height in meters). Based on the chart, the linear correlation between weight and BMI (roughly as BMI is also affected by height) makes that notion more evident. We can observe that most points are distributed evenly into most [*BMI categories*](https://en.wikipedia.org/wiki/Body_mass_index), from *Underweight Moderate thinness (BMI $16 - 16.9 kg/m^2$)* to *Obese Class II (BMI $35.0 - 39.9 kg/m^2$)*. This indicates the majority of population are unhealthy (most are out of the *Normal $18.5 - 24.9 kg/m^2$* range), which seems reasonable as the dataset we downloaded contains the measurements from *2021* to *2023*, the post-pandemic (COVID-19) period when food deficiency along with the closure of restaurants and gymnasiums were the main challenges that hindered people from staying healthy. We also see that the lowest and highest BMI values are $11.1$ ($< 16$ - Underweight Severe thinness) and $74.8$ ($>= 40$ - Obese Class III) respectively, which dictate either concerning health statuses or outliers from false measurements. It is worth to mention that the missing BMI values were imputed with the median BMI, which is $26.4$ (the horizontal line on the left of the chart). This helps us realise that most of the missing values are associated with points from $2.7kg$ to $15kg$, implying people in very young ages (including infants and toddlers), which were difficult to measure precisely and thus resulted in missing data; also the majority of people have BMI measurements around $26.4$ (median BMI), which belongs to the *Overweight* category and implies unhealthy population. Regardless of the insights that these imputed values have given, they are wrong and should not be taken into health reports. To summarise, from our observations, the United States governments need to adopt and implement new policies to narrow the BMI range down to *Normal*, enhancing the overall health of the U.S population.


## Column chart showing responses to the question "Blood ever tested for HIV virus?"

* **Idea:** Our idea for this section is a column chart illustrating the number of people going for every answer (*Yes*, *No*, *Refused*, *Don't know*) when asked the question *Blood ever tested for HIV virus?* - the [*HSQ590*](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/HSQ_L.htm#HSQ590) variable in the *Current Health Status* dataset. We can move the chart around, zoom in/out, download it, hover on each column to see the number of responses for that column, and can adjust the width of each column using the slider.

* In the original dataset, *HSQ590* contains numbers *1, 2, 7, 9* representing values *Yes*, *No*, *Refused*, *Don't know* respectively. For visualising purposes, leaving these values in their numerical form is meaningless, thus conversion to their corresponding categorical values is necessary, which can be done with the `.replace({old: new})` method, particularly `main_df["HSQ590"].replace({1: "Yes", 2: "No", 7: "Refused", 9: "Don't know"})`, and declared as categorical-type values `.astype(pd.CategoricalDtype(categories=["Yes", "No", "Refused", "Don't know"], ordered=True))` (`ordered=True` indicates that we want to keep the column ordering as in `categories`). We overwrite the original column with the converted column (`main_df["HSQ590"]`).

* We have got the column in the desired format, and now we need to get the count of each answer (count of each unique value in the column). We can use `main_df.groupby("HSQ590")` to group the values into their unique groups (with `HSQ590` column not as an index column - `as_index=False`, and `observed=True` excludes the values that are not in `categories`, but we use them all so this is just to silence the warning). Then we apply the `.size()` method to count the instances of each group, this adds the `size` column next to `HSQ590` column in the `HSQ590_df` sub data frame.

* For the creation of the chart, we first create a figure `col_chart` with the `figure()` function (imported from `bokeh.plotting`) with the title (`title="Responses to 'Blood ever tested for HIV virus?'"`), the x-axis title (`x_axis_label="Answers"`) and y-axis title (`y_axis_label="Number of people"`), the dimension *600px x 400px* (`width=600, height=400`), and the unique answers as the x-axis labels - `x_range=list(HSQ590_df["HSQ590"])` (the unique values in the `HSQ590` column of our sub data frame `HSQ590_df`). Then, we use `col_chart.vbar()` to add the vertical bars - with each has a width of *0.5* (`width=0.5`); `source=ColumnDataSource(HSQ590_df)` converts the sub data frame `HSQ590_df` into a format that fits `bokeh` (each column in the data frame is a key-value pair where key is represented by column name and value is an array of records stored in that column); from `source`, the unique answers (`HSQ590` column of `HSQ590_df`) are shown on the x-axis (`x="HSQ590"`) and the counts (`size` column of `HSQ590_df`) on y-axis (`top="size"`). The outcome of `col_chart.vbar()` is assigned to `col_renderer`.

* The next step is creating the slider (`slider`) for changing the width of columns (leveraging the imported `Slider` from `bokeh.models`). The range of this slider is *[0.1, 1]*, thus `start=0.1, end=1`; each step the slider changes *0.1* (`step=0.1`); the initial value *0.5* (`value=0.5`) as it is also the default we set for the chart when we created it; the name above this slider (`title="Width"`).

* We arrange the created components with the imported `column` from `bokeh.layouts` - `column(slider, col_chart)` puts `slider` above `col_chart`. The whole group is assigned to `layout`.

* Our chart also has the default interactive tools from `bokeh` as the previous chart (*Pan, Box Zoom, Wheel Zoom, Save, Reset*), and we also want to add a hover tool that displays the information of a column (in a placeholder) when we hover on it. Our object is `hover_tool`, created using `HoverTool` model from `bokeh.models` with `tooltips` parameter receives a list of information to be displayed - the string `"Answer"` followed by the actual answer of that associated column (`("Answer", "@HSQ590")`), and the string `"People"` along with the number of people choosing the answer (`("People", "@size")`). We then add our `hover_tool` to the chart (`col_chart.add_tools(hover_tool)`).

* The second interactive function that we are going to implement is changing the width of each column (using the slider). We have created the slider, but have yet to link it to our chart, and to do that, we must first create a connection named `update_col_chart`, which is the `CustomJS` method from `bokeh.models`. In `CustomJS()`, we must write a *JavaScript* code that performs our desired logic, consisting of 2 parts. For the first part, we map (pass) the variables created in *Python* (`col_renderer`, `slider`) to *JavaScript* through the dictionary of `args=dict()` (we keep the same name for every variable between 2 languages to avoid confusion). For the second part, we write *JavaScript* code inside `code=""" """`. To change width, we access `col_renderer.glyph` (the properties of `.vbar()` - `col_renderer`), then `.width` (refers to the width property of columns) that we set to the new width indicated by the slider's current value (`slider.value`).

* Now, we can declare the connection from `slider` to `col_renderer` through `js_on_change()` method with `"value"` representing the change in the value of `slider`, and it uses the logic `update_col_chart`. This means every time the slider changes, it executes the code in `update_col_chart`.

* Lastly, we use `show()` (from imported `bokeh.plotting`) to display our arranged `layout` (`show(layout)`).

In [23]:
# Converting values of HSQ590 to categorical type
main_df["HSQ590"] = (main_df["HSQ590"]
                     .replace({1: "Yes", 2: "No", 7: "Refused", 9: "Don't know"})
                     .astype(
                         pd.CategoricalDtype(
                             categories=["Yes", "No", "Refused", "Don't know"],
                             ordered=True)
                     )
                    )

# Data frame for the chart
HSQ590_df = main_df.groupby("HSQ590", as_index=False, observed=True).size()

# Create column chart
col_chart = figure(title="Responses to 'Blood ever tested for HIV virus?'",
                   x_axis_label="Answers", y_axis_label="Number of people",
                   width=600, height=400,
                   x_range=list(HSQ590_df["HSQ590"])
                  )
col_renderer = col_chart.vbar(width=0.5, source=ColumnDataSource(HSQ590_df),
                              x="HSQ590", top="size")

# Slider to control width
slider = Slider(start=0.1, end=1, step=0.1, value=0.5, title="Width")

# Arrange the components
layout = column(slider, col_chart)

# Add hover tool
hover_tool = HoverTool(tooltips=[
    ("Answer", "@HSQ590"),
    ("People", "@size")
])
col_chart.add_tools(hover_tool)

# CustomJS logic to dynamically change bars' width
update_col_chart = CustomJS(args=dict(col_renderer=col_renderer, slider=slider),
                            code="""
                                 col_renderer.glyph.width = slider.value;
                                 """)

# Call the implemented logic whenever the slider is changed
slider.js_on_change("value", update_col_chart)

# Display using show(layout)

In [24]:
show(layout)


$\rightarrow$ From the chart, the majority response is "No", which has been chosen by 7196 people. This overwhelms the other categories ("Yes" with 1654 people, "Don't know" with 9 people, and "Refused" with 1 person). This data raises concerns about people's HIV awareness and the tendency of them paying little attention to their health. However, considering the context of post-pandemic period, it is not solely the people themselves. The COVID-19 pandemic has redirected global healthcare to focus on preventing the number of positive cases from rising and inadvertently neglecting HIV causes. In particular, many healthcare facilities have prioritised COVID-19 testing over HIV screening in terms of medical devices and rooms for testing, resulting in lower HIV testing rates. Additionally, people were also encouraged to stay in quarantine, especially those who tested positive for the disease (F0) and their surroundings (F1, F2), so going outside to do routine HIV check-ups was not the top priority at that time.


## Line chart illustrating average Energy generated (measured on first day) for each day of the week

* **Idea:** Our third visualisation is a line chart depicting the average energy generated from foods and beverages ([DR1TKCAL](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/DR1TOT_L.htm#DR1TKCAL)) recorded on the first day after 24 hours for each day of the week ([DR1DAY](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/DR1TOT_L.htm#DR1DAY)). These are variables of the [*Dietary Interview - Total Nutrient Intakes, First Day*](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/DR1TOT_L.htm) dataset. We can move the chart around, zoom in/out, download it, hover on each point to see the energy value of the corresponding day, and can dynamically adjust the *RGB* colors of the line using the sliders.

* In the original dataset, *DR1DAY* contains numbers *1, 2, 3, 4, 5, 6, 7* representing the days *Sunday, Monday, Tuesday, Wednesday, Thursday, Friday, Saturday* respectively. For visualising purposes, leaving these values in their numerical form is meaningless, thus converting to their corresponding categorical values is necessary, which can be done with the `.replace({old: new})` method, particularly `main_df["DR1DAY"].replace({1: "Sunday", 2: "Monday", 3: "Tuesday", 4: "Wednesday", 5: "Thursday", 6: "Friday", 7: "Saturday"})`, and declared as categorical-type values `.astype(pd.CategoricalDtype(categories=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], ordered=True))` (`ordered=True` indicates that we want to keep the column ordering as in `categories`). We overwrite the original column with the converted column (`main_df["DR1DAY"]`).

* We have got the column in the desired format, and now we want to group the energy values (`DR1TKCAL`) by day (`DR1DAY`), which is implemented through `main_df.groupby("DR1DAY")` (with `DR1DAY` column not as an index column - `as_index=False`, and `observed=True` excludes the values that are not in `categories`, but we use them all so this is just to silence the warning). Then we can calculate the average energy for each day by applying `["DR1TKCAL"].mean()` (essentially computing the average grouped energy values `DR1TKCAL` for each unique group `DR1DAY`). The newly generated sub data frame is assigned to `energy_df`.

* For the creation of the chart, we first create a figure `line_chart` with the `figure()` function (imported from `bokeh.plotting`) with the title (`title="Average Energy generated for each day of the week"`), the x-axis title (`x_axis_label="Days of week"`) and y-axis title (`y_axis_label="Energy (kcal)"`), the dimension *600px x 400px* (`width=600, height=400`), and the days as the x-axis labels - `x_range=list(energy_df["DR1DAY"])` (the unique values in the `DR1DAY` column of our sub data frame `energy_df`). Then, we use `line_chart.line()` to draw the line segments - with each has a width of *3* (`line_width=3`); black as default color (`color="black"`); `source=ColumnDataSource(energy_df)` converts the sub data frame `energy_df` into a format that fits `bokeh` (each column in the data frame is a key-value pair where key is represented by column name and value is an array of records stored in that column); from `source`, the days (`DR1DAY` column of `energy_df`) are shown on the x-axis (`x="DR1DAY"`) and the computed average values (`DR1TKCAL` column of `energy_df`) on y-axis (`y="DR1TKCAL"`). The outcome of `line_chart.line()` is assigned to `line_renderer`.

* The next step is creating the 3 sliders `sliderR`, `sliderG`, `sliderB` representing the *Red, Green, BLue* colors respectively (utilising the imported `Slider` from `bokeh.models`). The color spectrum of each slider is *[0, 255]*, thus `start=0, end=255`; for each step the sliders change *1* (`step=1`); the initial values are all *0* (`value=0`) as it is also the default settings we set for the chart when we created it (*black color* corresponds to *Red = Green = Blue = 0*); the respective names above the sliders `sliderR`, `sliderG`, `sliderB` are `title="Red"`, `title="Green"`, `title="Blue"`.

* We arrange the created components with the imported `row` and `column` from `bokeh.layouts` - `row(sliderR, sliderG, sliderB)` generates a row with `sliderR`, `sliderG`, `sliderB` from left to right. This row is placed on the top of `line_chart`, so they are put in a column (`column()`). The whole group is assigned to `layout`.

* Our chart also has the default interactive tools from `bokeh` as the two previous charts (*Pan, Box Zoom, Wheel Zoom, Save, Reset*), and we also want to add a hover tool that displays the information of a point (in a placeholder) when we hover on it. Our object is `hover_tool`, created using `HoverTool` model from `bokeh.models` with `tooltips` parameter receives a list of information to be displayed - the string `"Day"` followed by the actual day of that associated point (`("Day", "@DR1DAY")`), and the string `"Energy (kcal)"` along with the average energy value of the corresponding day (`("Energy (kcal)", "@DR1TKCAL{0.00}")` - `{0.00}` converts the value to the number format with 2 decimal places). We then add our `hover_tool` to the chart (`line_chart.add_tools(hover_tool)`).

* The second interactive function that we are going to implement is changing the line color using sliders. We have created the sliders, but have yet to link them to our chart, and to do that, we must first create a connection named `update_line_chart`, which is the `CustomJS` method from `bokeh.models`. In `CustomJS()`, we must write a *JavaScript* code that performs our desired logic, consisting of 2 parts. For the first part, we map (pass) the variables created in *Python* (`line_renderer`, `sliderR`, `sliderG`, `sliderB`) to *JavaScript* through the dictionary of `args=dict()` (we keep the same name for every variable between 2 languages to avoid confusion). For the second part, we write *JavaScript* code inside `code=""" """`. To change the color, we access `line_renderer.glyph` (the properties of `.line()` - `line_renderer`), then `.line_color` (refers to the line color property of the line chart) that we set to the new color indicated by the combination of 3 sliders' current values (``rgb(${sliderR.value}, ${sliderG.value}, ${sliderB.value})``).

* Now, we can declare the connection from `sliderR`, `sliderG`, `sliderB` to `line_renderer` through `js_on_change()` method with `"value"` representing the change(s) in the value(s) of the slider(s), and they all use the logic `update_line_chart`. This means every time the slider(s) change, the code in `update_line_chart` is executed.

* Lastly, we use `show()` (from imported `bokeh.plotting`) to display our arranged `layout` (`show(layout)`).

In [27]:
# Converting values of DR1DAY to categorical type
main_df["DR1DAY"] = (main_df["DR1DAY"]
                     .replace({1: "Sunday", 2: "Monday", 3: "Tuesday", 4: "Wednesday",
                              5: "Thursday", 6: "Friday", 7: "Saturday"})
                     .astype(
                         pd.CategoricalDtype(
                             categories=["Monday", "Tuesday", "Wednesday", "Thursday",
                                         "Friday", "Saturday", "Sunday"],
                             ordered=True)
                     )
                    )

# Data frame for the chart
energy_df = (main_df
             .groupby("DR1DAY", as_index=False, observed=True)
             ["DR1TKCAL"]
             .mean()
            )

# Create line chart
line_chart = figure(title="Average Energy generated for each day of the week",
                    x_axis_label="Days of week",
                    y_axis_label="Energy (kcal)",
                    width=600, height=400,
                    x_range=list(energy_df["DR1DAY"])
                   )
line_renderer = line_chart.line(line_width=3, color="black",
                                source=ColumnDataSource(energy_df),
                                x="DR1DAY", y="DR1TKCAL"
                               )

# Slider to control color
sliderR = Slider(start=0, end=255, step=1, value=0, title="Red")
sliderG = Slider(start=0, end=255, step=1, value=0, title="Green")
sliderB = Slider(start=0, end=255, step=1, value=0, title="Blue")

# Arrange components
layout = column(
                row(sliderR, sliderG, sliderB),
                line_chart
               )

# Add hover tool
hover_tool = HoverTool(tooltips=[
    ("Day", "@DR1DAY"),
    ("Energy (kcal)", "@DR1TKCAL{0.00}")
])
line_chart.add_tools(hover_tool)

# CustomJS logic to dynamically change line's color
update_line_chart = CustomJS(args=dict(line_renderer=line_renderer,
                              sliderR=sliderR, sliderG=sliderG, sliderB=sliderB),
                    code="""
line_renderer.glyph.line_color = `rgb(${sliderR.value}, ` +  
                                 `${sliderG.value}, ${sliderB.value})`;
""")

# Call the implemented logic whenever the slider(s) is/are changed
sliderR.js_on_change("value", update_line_chart)
sliderG.js_on_change("value", update_line_chart)
sliderB.js_on_change("value", update_line_chart)

# Display using show(layout)

In [28]:
show(layout)


$\rightarrow$ An interesting pattern can be observed from this chart, that is the energy generated from intaking foods and beverages is primarily low in the middle of the week, but significantly high in the weekend, especially for Saturday. This can be explained by the fact that in midweek, people did not have much time to eat since they were busy studying or working, and some might skip their meals as they wanted to be punctual for classes or meetings, not to mention the intense responsibilities at studying (difficult lessons, exams, grades) or working (designing ideas, managing meetings, running projects). These factors drained people's energy and an evident observation is the average energy dropped to the bottom on Wednesday, with only *1834.58 kcal*. Nevertheless, from that day onwards, the energy rose gradually and reached its peak on Saturday (*2027.13 kcal*), perhaps because people were more excited towards the weekend break. In contrast to midweek, Saturday was the most comfortable day of the week as students and workers entered their first day of the weekend break, alleviating midweek stress and facilitating eating out with family and friends more. The post-COVID period was when restrictions eased so people resumed their socialising and dining out habits on the first day of weekend break, and eating out involved large portions and calorically dense foods, demonstrating the spike in energy intake. However, Sunday, the second day of the weekend was when people had to prepare for the upcoming week of studying or working, so the energy level plummeted.


## Distribution of Total Cholesterol through Histogram

* **Idea:** Our fourth visualisation is a histogram showing the distribution of *Total Cholesterol* ([LBDTCSI](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/TCHOL_L.htm#LBDTCSI)) - a variable in *Cholesterol - Total* dataset. We can move the chart around, zoom in/out, download it, hover on each interval to see the number of observations in that interval, and can adjust the color of the chart using a dropdown menu.

* For this histogram, we just simply get all the values in the `LBDTCSI` column of `main_df` and convert these into a list (`main_df["LBDTCSI"].tolist()`) named `cholesterol_list`.

* For the creation of the chart, we first create a figure `histogram` with the `figure()` function (imported from `bokeh.plotting`) with the title (`title="Distribution of total cholesterol"`), the x-axis title (`x_axis_label="Total cholesterol (mmol/L)"`) and y-axis title (`y_axis_label= "Frequency"`), the dimension *600px x 400px* (`width=600, height=400`), and toolbar with tools *Pan, Box Zoom, Wheel Zoom, Save, Reset* on the right (`toolbar_location="right"`). Next, we create `bins` (intervals) for the histogram using `np.linspace(min(cholesterol_list), max(cholesterol_list), 21)` (the range from the smallest value `min(cholesterol_list)` to the largest value `max( cholesterol_list)`, and divided into `21` boundaries, which is equivalent to *20* intervals). Then, `np.histogram( cholesterol_list, bins=bins)` distributes each cholesterol value in `cholesterol_list` to the interval (`bins=bins`) it belongs to, counts the frequency of each interval, and returns the list of frequencies (`frequencies`) as well as the list of boundaries (`boundaries`) that separate the intervals. Having enough resources to draw the histogram, we proceed with `histogram.quad()` - `top=frequencies` specifies the heights of the bars as the frequencies; `bottom=0` starts each bar from *0*; `left=boundaries[:-1]`/`right=boundaries[1:]` defines the left/right edges for the bars as the boundaries except the last/first one; the default fill color and border color of the bars are both *purple* (`fill_color="purple", line_color="purple"`). The outcome of `histogram.quad()` is assigned to `hist_renderer`.

* The next step is to create a dropdown menu (`dropdown`) with a few colors as options for the histogram's color (utilising the imported `Dropdown` from `bokeh.models`, an alternative of `Select`). This has *Select color* as the label shown inside it (`label="Select color"`), and `menu` receives a list of tuples where each consists of the label shown as an option in the dropdown menu (e.g. `"Red"`) and the actual argument itself (e.g. `"red"` - without capitalising the first letter). Here we have 6 colors *Red, Blue, Purple, Orange, Brown, Black*, so `menu=[("Red", "red"), ("Blue", "blue"), ("Purple", "purple"), ("Orange", "orange"), ("Brown", "brown"), ("Black", "black")]`.

* We then arrange the created components with the imported `column` from `bokeh.layouts` - `column(dropdown, histogram)` puts `dropdown` above `histogram`. The whole group is assigned to `layout`.

* Besides the default tools on the toolbar (*Pan, Box Zoom, Wheel Zoom, Save, Reset*), and we also want to add a hover tool that displays the information of an interval (in a placeholder) when we hover on it. Our object is `hover_tool`, created using `HoverTool` model from `bokeh.models` with `tooltips` parameter receives a list of information to be displayed - the string `"Range"` followed by the actual range of that associated interval (`("Range", "@left to @right")` - these are `left` and `right` boundaries of each bar when we created `histogram.quad()`), and the string `"Frequency"` along with the frequency of the corresponding interval (`("Frequency", "@top")` - another parameter `top` in the created `histogram.quad()`). We then add our `hover_tool` to the chart (`histogram.add_tools(hover_tool)`).

* The second interactive function that we are going to implement is selecting the histogram color from the dropdown menu. We have created the dropdown menu, but have yet to link it to our histogram, and to do that, we must first create a connection named `update_histogram`, which is the `CustomJS` method from `bokeh.models`. In `CustomJS()`, we must write a *JavaScript* code that performs our desired logic, consisting of 2 parts. For the first part, we map (pass) variable `hist_renderer` created in *Python* to *JavaScript* through the dictionary of `args=dict()` (we keep the same name for the variable between 2 languages to avoid confusion). For the second part, we write *JavaScript* code inside `code=""" """`. To change the color, we access `hist_renderer.glyph` (the properties of `.quad()` - `hist_renderer`), then `.fill_color` and `.line_color` (refer to the fill color and line color properties of the histogram) that we both set to the new color indicated by the selected dropdown menu's value (`cb_obj.item` - the selected `item` of the dropdown menu `cb_obj`).

* Now, we can declare the connection from `dropdown` to `hist_renderer` through `js_on_event()` method with `"menu_item_click"` representing when an `item` from `dropdown` is clicked, and it uses the logic `update_histogram`. This means every time the value of `dropdown` changes, the code in `update_histogram` is executed.

* Lastly, we use `show()` (from imported `bokeh.plotting`) to display our arranged `layout` (`show(layout)`).

In [31]:
# Data
cholesterol_list = main_df["LBDTCSI"].tolist()

# Histogram
histogram = figure(title="Distribution of total cholesterol",
                   x_axis_label="Total cholesterol (mmol/L)",
                   y_axis_label="Frequency",
                   width=600, height=400, toolbar_location="right"
                  )

bins = np.linspace(min(cholesterol_list), max(cholesterol_list), 21)
frequencies, boundaries = np.histogram(cholesterol_list, bins=bins)
hist_renderer = histogram.quad(top=frequencies, bottom=0,
                               left=boundaries[:-1], right=boundaries[1:],
                               fill_color="purple", line_color="purple")

# Dropdown menu for color options
dropdown = Dropdown(label="Select color",
                    menu=[("Red", "red"), ("Blue", "blue"),
                          ("Purple", "purple"), ("Orange", "orange"),
                          ("Brown", "brown"), ("Black", "black")]
                   )

# Arrange components
layout = column(dropdown, histogram)

# Add hover tool
hover_tool = HoverTool(tooltips=[("Range", "@left to @right"),
                                 ("Frequency", "@top")]
                      )
histogram.add_tools(hover_tool)

# CustomJS logic to change histogram's color
update_histogram = CustomJS(args=dict(hist_renderer=hist_renderer),
                            code="""
    hist_renderer.glyph.fill_color = cb_obj.item;
    hist_renderer.glyph.line_color = cb_obj.item;
""")

# Call the implemented logic whenever a new value of dropdown menu is selected
dropdown.js_on_event("menu_item_click", update_histogram)

# Display using show(layout)

In [32]:
show(layout)


$\rightarrow$ A few observations can be derived through this distribution. The most common range for total cholesterol is *4.519 to 5.006 mmol/L* with *3209 people* falling into this interval. Assessing a wider range, *from 4.033 to 5.492 mmol/L*, *5296 people* belong to this, which implies that the majority of the population have healthy blood cholesterol levels since the [healthy range is *between 3.9 and 5.5 mmol/L*](https://www.victorchang.edu.au/heart-disease/high-cholesterol). This histogram is also right-skewed as the right tail is longer than the left one, indicating either imprecise measurements leading to outliers, or a few people have excessive cholesterol levels. To be specific, *1484* people have their cholesterol values exceeded the normal range, with a person falling into the range *10.844 - 11.33 mmol/L*. Thus, while the majority have healthy blood cholesterol levels, it is imperative to pay more attention to the people outside of the range (including people with lower cholesterol levels than expected - about *2080* people), especially after the pandemic, the period when the focus was on COVID-19 tests with medical devices and rooms prioritised for them, neglecting the needs for other aspects of health such as blood cholesterol level or HIV disease as analysed in our second visualisation.


## Heat map visualising Pearson's correlation matrix of Blood Pressure pairs

* **Idea:** For the last visualisation, we design a heat map illustrating Pearson's correlation coefficient for every pair of variables in [*Blood Pressure*](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/BPXO_L.htm) dataset. We can move the heat map around, zoom in/out, download it, hover on each cell to see the correlation coefficient of the corresponding pair, and can adjust the color of the chart using a dropdown menu.

* Since only the numerical variables are involved in the correlation matrix, we extract them from `main_df` (`main_df[["BPXOSY1", "BPXODI1", "BPXOSY2", "BPXODI2", "BPXOSY3", "BPXODI3", "BPXOPLS1", "BPXOPLS2", "BPXOPLS3"]]`). The numerical variables are originally in code form (`BPXOSY1`, `BPXODI1`, `BPXOSY2`, `BPXODI2`, `BPXOSY3`, `BPXODI3`, `BPXOPLS1`, `BPXOPLS2`, `BPXOPLS3`), and for a more meaningful presentation, we change these to their associated names (as per this [Codebook](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/BPXO_L.htm#Codebook)) by `.rename(columns={"BPXOSY1":"1st Systolic", "BPXODI1": "1st Diastolic", "BPXOSY2": "2nd Systolic", "BPXODI2": "2nd Diastolic", "BPXOSY3": "3rd Systolic", "BPXODI3": "3rd Diastolic", "BPXOPLS1": "1st Pulse", "BPXOPLS2": "2nd Pulse", "BPXOPLS3": "3rd Pulse"})`, and store them in a sub data frame `bloodpressure_df`.

* Next, we generate the correlation matrix `corr_matrix` (compute correlation coefficients) for our sub data frame using `bloodpressure_df.corr()`. Since `bokeh` only accepts the matrix in the *long* format (rows which each containing the variables in the pair and their correlation coefficient), we can do the conversion with `.stack()` (which returns a Series), so `.reset_index()` converts back to the data frame, and we assign this to `corr_long`. We must also rename the columns in `corr_long` so each row contains pair `x, y` and that pair's coefficient `coeff` (`corr_long.columns = ["x", "y", "coeff"]`).

* For the creation of the chart, we first create a figure `heatmap` with the `figure()` function (imported from `bokeh.plotting`) with the title (`title="Pearson's correlation coefficients of Blood Pressure pairs"`), toolbar with tools *Pan, Box Zoom, Wheel Zoom, Save, Reset* on the right (`toolbar_location="right"`), the dimension *510px x 400px* (`width=510, height=400` - the heatmap's shape is square but we have to leave out extra space for the color bar and the toolbar), the x-axis labels are placed below the heatmap (`x_axis_location="below"`), the matrix's column labels as the x-axis labels - `x_range=list(corr_matrix.columns)` (those are `bloodpressure_df`'s columns before we inputted into calculating `corr_matrix`), and the same goes for y-axis labels but we also have to reverse the order since we want the top-down order and the default y-coordinate system goes bottom-up (`y_range=list(corr_matrix.columns[::-1])`). To avoid overlapping x-axis labels, we rotate them to the left by 30 degrees by accessing `heatmap.xaxis.major_label_orientation` and setting it to the radian value of 30 degrees (`30 * (3.14 / 180)`). Thereafter, we use a `color_mapper` to map the calculated coefficients into their corresponding colors, this leverages the `LinearColorMapper` method from `bokeh.models` with the default palette `palette="Magma256"`, *[-1, 1]* as the range of coefficients (`low=-1, high=1`). Subsequently, we use `heatmap.rect()` to draw the cells - with each has a *1x1* dimension (`width=1, height=1`); no lines are shown (`line_color=None`); `source=ColumnDataSource(corr_long)` converts the sub data frame `corr_long` into a format that fits `bokeh` (each column in the data frame is a key-value pair where key is represented by column name and value is an array of records stored in that column); from `source`, the `x` variables (`x` column of `corr_long`) are used on the x-axis (`x="x"`) and the `y` variables (`y` column of `corr_long`) on y-axis (`y="y"`), the mapping of each coefficient and its color (`color_mapper`) is the cell color (`fill_color=transform("coeff", color_mapper)` - utilising `transform` from the imported `bokeh.transform`). The outcome of `heatmap.rect()` is assigned to `heatmap_renderer`. We also want to add the color bar (`color_bar`) as a legend to guide us on the color ranges that indicate large and small values; this uses `ColorBar()` from `bokeh.models` with the created mapping `color_mapper=color_mapper` and the shown label `title="Pearson's coefficients"`. We add this `color_bar` to the right of our `heatmap` (`heatmap.add_layout(color_bar, "right")`), remember that we set figure's width to be larger than its height to accommodate this.

* The next step is to create a dropdown menu (`dropdown`) with a few palettes as options for the heatmap (utilising the imported `Dropdown` from `bokeh.models`, an alternative of `Select`). This has *Choose palette* as the label shown inside it (`label="Choose palette"`), and `menu` receives a list of tuples where each consists of the label shown as an option in the dropdown menu (e.g. `"Inferno"`) and the actual argument itself (e.g. `"Inferno"`). Here we have 5 palettes *Cividis, Magma, Inferno, Plasma, Viridis*, so `menu=[("Cividis", "Cividis"), ("Magma", "Magma"), ("Inferno", "Inferno"), ("Plasma", "Plasma"), ("Viridis", "Viridis")]`.

* We then arrange the created components with the imported `column` from `bokeh.layouts` - `column(dropdown, heatmap)` puts `dropdown` above `heatmap`. The whole group is assigned to `layout`.

* Besides the default tools on the toolbar (*Pan, Box Zoom, Wheel Zoom, Save, Reset*), and we also want to add a hover tool that displays the information of a cell (in a placeholder) when we hover on it. Our object is `hover_tool`, created using `HoverTool` model from `bokeh.models` with `tooltips` parameter receives a list of information to be displayed - the string `"Pair"` followed by the actual pair of that associated cell (`("Pair", "@x, @y")` - these are `x` and `y` variables when we created `heatmap.rect()`), and the string `"Coefficient"` along with the coefficient value of the corresponding cell (`("Coefficient", "@coeff")` - another parameter `coeff` in the created `heatmap.rect()`). We then add our `hover_tool` to the chart (`heatmap.add_tools(hover_tool)`).

* Now for the choosing palettes part, we need to create the `palettes` dictionary that will be used in our dropdown menu's logic. For each palette in the list of all palettes (`for palette in ['Cividis', 'Magma', 'Inferno', 'Plasma', 'Viridis']`), we add a key-value pair to our dictionary. The pair has the name of the palette (`palette`) is the key; for the value, `f"bokeh.palettes.{palette}"` accesses the palette object with the name `palette` in `bokeh.palettes` module, then `eval()` evaluates that object and returns a dictionary containing "index - list of colors" pairs, so `.values()` returns all lists of colors in that dictionary, and `[-1]` retrieves the list with most colors (we want our color spectrum to be wide). Thus, `palettes` is a dictionary in which each key-value pair has the key represented by a palette's name and the value represented by the list of all colors in that palette.

* After everything is properly constructed, we create the logic named `update_heatmap`, which is the `CustomJS` method from `bokeh.models`. In `CustomJS()`, we must write a *JavaScript* code that performs our desired logic, consisting of 2 parts. For the first part, we map (pass) the variables `heatmap_renderer`, `palettes`, `color_mapper` created in *Python* to *JavaScript* through the dictionary of `args=dict()` (we keep the same name for each variable between 2 languages to avoid confusion). For the second part, we write *JavaScript* code inside `code=""" """`. To change the palette, we access `heatmap_renderer.glyph` (the properties of `heatmap.rect()` - `heatmap_renderer`), then `.fill_color` (refers to the `fill_color` property of the created `heatmap.rect()`), then `.transform` (in `fill_color` of `heatmap.rect()`), and since the `transform` object that we created has the mapping `color_mapper` that uses `LinearColorMapper()` with the parameter `palette`, we access the last object in our path that is `.palette` (the `palette` property of our heatmap). We set the palette property to the list of colors that corresponds to the selected palette in the dropdown menu (`palettes[cb_obj.item]` - accessing the `palettes` dictionary with the selected `item` of the dropdown menu `cb_obj` as the key, which returns the value of that associated key or the list of colors of that corresponding `cb_obj.item`).

* Now, we can declare the connection from `dropdown` to `heatmap_renderer` through `js_on_event()` method with `"menu_item_click"` representing when an `item` from `dropdown` is clicked, and it uses the logic `update_heatmap`. This means every time the value of `dropdown` changes, the code in `update_heatmap` is executed.

* Lastly, we use `show()` (from imported `bokeh.plotting`) to display our arranged `layout` (`show(layout)`).

In [35]:
# Sub data frame
bloodpressure_df = (main_df
                    [["BPXOSY1", "BPXODI1", "BPXOSY2", "BPXODI2", "BPXOSY3",
                            "BPXODI3", "BPXOPLS1", "BPXOPLS2", "BPXOPLS3"]]
                    .rename(columns={"BPXOSY1":"1st Systolic",
                                     "BPXODI1": "1st Diastolic",
                                     "BPXOSY2": "2nd Systolic",
                                     "BPXODI2": "2nd Diastolic",
                                     "BPXOSY3": "3rd Systolic",
                                     "BPXODI3": "3rd Diastolic",
                                     "BPXOPLS1": "1st Pulse",
                                     "BPXOPLS2": "2nd Pulse",
                                     "BPXOPLS3": "3rd Pulse"})
                   )

# Pearson's correlation matrix
corr_matrix = bloodpressure_df.corr()

# Convert correlation matrix to long format for bokeh
corr_long = corr_matrix.stack().reset_index()
corr_long.columns = ["x", "y", "coeff"]

# Create the heatmap figure
heatmap = figure(
    title="Pearson's correlation coefficients of Blood Pressure pairs",
    toolbar_location="right",
    width=510, height=400,
    x_axis_location="below",
    x_range=list(corr_matrix.columns),
    y_range=list(corr_matrix.columns[::-1]),    
)

# Rotate x-axis labels by 30 degrees
heatmap.xaxis.major_label_orientation = 30 * (3.14 / 180)

# Color mapper to map the coefficients to colors
color_mapper = LinearColorMapper(palette="Magma256", low=-1, high=1)

# Draw the cells
heatmap_renderer = heatmap.rect(
    width=1, height=1,
    line_color=None,
    source=ColumnDataSource(corr_long),
    x="x", y="y",
    fill_color=transform("coeff", color_mapper)
)

# Add color bar
color_bar = ColorBar(color_mapper=color_mapper, title="Pearson's coefficients")
heatmap.add_layout(color_bar, "right")

# Dropdown menu for choosing palettes
dropdown = Dropdown(label="Choose palette",
                    menu=[("Cividis", "Cividis"), ("Magma", "Magma"),
                          ("Inferno", "Inferno"), ("Plasma", "Plasma"),
                          ("Viridis", "Viridis")]
                   )

# Arrange components
layout = column(dropdown, heatmap)

# Add a hover tool
hover_tool = HoverTool(
    tooltips=[
        ("Pair", "@x, @y"),
        ("Coefficient", "@coeff")
    ]
)
heatmap.add_tools(hover_tool)

# Create a dictionary of palettes
palettes = {palette: list(eval(f"bokeh.palettes.{palette}").values())[-1]
            for palette in ['Cividis', 'Magma', 'Inferno', 'Plasma', 'Viridis']}

# CustomJS logic to change heatmap's palette
update_heatmap = CustomJS(
    args=dict(heatmap_renderer=heatmap_renderer,
              palettes=palettes, color_mapper=color_mapper),
    code="""
    heatmap_renderer.glyph.fill_color.transform.palette = palettes[cb_obj.item];
    """
)

# Call the implemented logic whenever a new palette from dropdown menu is selected
dropdown.js_on_event("menu_item_click", update_heatmap)

# Display using show(layout)

In [36]:
show(layout)


$\rightarrow$ Our heat map is symmetrical about the diagonal (from the top left cell to the bottom right cell), which means we only need to look at either the cells from the diagonal and above or from it and below, and we choose the above. Some intriguing observations can be drawn with the color of these cells. The diagonal contains cells where each correlates a variable on the x-axis with itself on the y-axis, so it is self-explanatory that these cells all have the maximum correlation coefficient (*1* - brightest color). Moving to cells with less bright colors, we see that cells correlating Systolic at different readings (1st, 2nd, and 3rd) have values just below *1*, with consecutive readings (1st and 2nd with *0.933*, 2nd and 3rd with *0.934*) having little higher correlation values than 1st and 3rd with *0.924*; the same pattern goes for Diastolic (1st and 2nd with *0.905*, 2nd and 3rd with *0.912*, 1st and 3rd with *0.898*) and Pulse (1st and 2nd with *0.948*, 2nd and 3rd with *0.947*, 1st and 3rd with *0.933*). It is worth mentioning that every 2 consecutive readings were taken 60 seconds apart (according to the [data documentation of the original dataset](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/BPXO_L.htm#Protocol_and_Procedure)), so they did not vary much and so they have greater correlation coefficients compared to the 120-second readings (1st and 3rd). The darker cells show the correlations between Systolic and Diastolic readings. Out of these values, the measurements at the same readings have highest correlated coefficients (1st Systolic and 1st Diastolic with *0.659*, 2nd Systolic and 2nd Diastolic with *0.653*, 3rd Systolic and 3rd Diastolic with *0.652*). The greater coefficients with consecutive readings are also shown in these cells (1st Systolic and 2nd Diastolic with *0.621*, 2nd Systolic and 3rd Diastolic with *0.619*, 1st Diastolic and 2nd Systolic with *0.644*), which are mostly greater than 120-second-apart readings (1st Systolic and 3rd Diastolic with *0.608*), except for a quite surprising result when 1st Diastolic and 3rd Systolic are correlated with a higher coefficient (*0.641*) than 2nd Diastolic and 3rd Systolic (*0.640*). From our observations, Diastolic and Systolic measurements are correlated in certain ways with readings of the same type related with values near *1* and different types with lower values but still greater than *0.5*; however, pairs involving Pulse have nearly no correlations as their coefficients are either near and above *0* like Pulse and Diastolic measurements (second darkest cells), or near and below *0* like Pulse and Systolic measurements (darkest cells). [Systolic blood pressure](https://www.heart.org/en/health-topics/high-blood-pressure/understanding-blood-pressure-readings) measures the pressure of blood pushing the artery walls when the heart beats, and practically, as the heart pumps blood more forcefully when it beats (raising Systolic pressure), the pressure against the artery walls when it rests, which is Diastolic blood pressure, also rises, and this explains the correlation between Systolic and Diastolic measures. Conversely, Pulse gives the number of times the heart beats per minute (bpm), and this has no direct involvement in the pressure on the artery walls (the heart beats fast does not result in higher pressure), thus Pulse has little to no relationship with either Systolic or Diastolic.


## Potential data privacy and ethics issues

### How data privacy and ethics issues arise

We have converted raw datasets into different visualisations and gained valuable insights on them, however, it is concerning that these might raise potential data breach issues. In particular, the downloaded datasets contain information from the survey participants, and while the identity of each individual is hidden by removing their name (or it was not included from the start), it can still be identified when gathering the pieces of information (connecting the attributes of a row). For instance, by assessing the [*Body Measures* dataset](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/BMX_L.htm), if we have an acquaintance's hip circumference, waist circumference, upper arm length (from elsewhere), we can probably search for the matching values in the table and find other measures of that person such as weight or height as they all belong to the same row. This becomes more dangerous if supported by data linkage, essentially the merging of different datasets, which is what we did in section 1.4, so despite the removal of names and the generation of *SEQN*s, these still serve as an identifier (each distinct *SEQN*) that facilitates linking between data of an individual. If we circle back to our example, assuming that we have data on all body measures of our acquaintance, and by using their *SEQN* as a tool to join *Body Measures* with other datasets (*Blood Pressure*, *Current Health Status*, *Diet*, *Total Cholesterol*), we will know which food they ate (e.g. [`DRD350H` in *Diet*](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/DR1TOT_L.htm#DRD350H)), what their heart rate was (e.g. [`BPXOPLS1` in *Blood Pressure*](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/BPXO_L.htm#BPXOPLS1)), what their total cholesterol was (e.g. [`LBDTCSI` in *Total Cholesterol*](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/TCHOL_L.htm#LBDTCSI)), and whether they had their blood tested for HIV (e.g. [`HSQ590` in *Current Health Status*](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/HSQ_L.htm#HSQ590)). In general, by merging different datasets using a unique identifier, we can derive any information from a few attributes of a person. This process is also called re-identification, an inverse of de-identification or anonymisation (removal of names).

### Consequences of re-identification

There are lots of consequences related to social aspects of re-identification. With our datasets, if a person's weight and height are known by others, they might face discriminations; if their heart rate, total cholesterol, and response to the HIV question are disclosed, they might be criticised on how badly they managed their health; or with the information about what food they ate, it is simply just embarrassment. Although these data do help the owners to monitor their health, if any of these are leaked, they will be at risk of being traumatised.

### Current data privacy laws and what needs to change

Currently, data privacy laws like [GDPR](https://gdpr-info.eu/) and [The Privacy Act 1988](https://www.oaic.gov.au/privacy/privacy-legislation/the-privacy-act) do address re-identification but only to a certain extent. GDPR allows "sufficient anonymisation" without explicitly stating what attributes should be removed to satisfy that, and the Privacy Act 1988 focuses on de-identification, not the prevention of re-identification. From our analysis in this section, it is a caution for these laws to be modified in such a way that directly resolves re-identification and prevents it exhaustively.

## Conclusion

In summary, we loaded our 5 **NHANES** datasets (*Body Measures*, *Current Health Status*, *Diet*, *Total Cholesterol*, *Blood Pressure*) in section 1.3 and merged them into a single data frame in the following section. We then replaced the missing data with appropriate values, declared output in the notebook, and started drawing the 5 interactive visualisations (each associated with a dataset - *Scatter chart visualising pairs of body measures* from *Body Measures*, *Column chart showing responses to the question "Blood ever tested for HIV virus?"* from *Current Health Status*, *Line chart illustrating average Energy generated (measured on first day) for each day of the week* from *Diet*, *Distribution of Total Cholesterol through Histogram* from *Total Cholesterol*, *Heat map visualising Pearson's correlation matrix of Blood Pressure pairs* from *Blood Pressure*) along with analysing observed patterns from those charts to draw valuable insights on practical settings related to health, the main purpose of this survey. In the final part, we addressed the concerns of data privacy from re-identification and the downsides of current data privacy laws.